# DQN Trading Agent

## Read database

In [1]:
#TO READ THE DATABASE
import h5py

DB_FILE = "../data/dataset_1h_1000.hdf5"
DB = h5py.File(DB_FILE, "r")

for item in DB:
    print(item)

bitcoin_usd
etherium_usd
ripple_usd


In [2]:
# Show the values of the first timestep
print(DB["bitcoin_usd"][0:2])

[[  1.51543080e+12   1.47930000e+04   1.46940000e+04   1.48600000e+04
    1.46690000e+04   7.86847895e+02]
 [  1.51542720e+12   1.44000000e+04   1.47890000e+04   1.49230000e+04
    1.43470000e+04   4.20724394e+03]]


## Define packages

In [3]:
import random
import numpy as np

## Define DQN Agent

In [4]:
from collections import deque
from keras.models import Model 
from keras.layers import Input, Dense, Flatten, Reshape
from keras.optimizers import Adam
from keras import backend as K
import keras

K.set_image_dim_ordering('tf')

class DQNAgent:
    def __init__(self, frame_size, frame_parameter_number, aux_number, action_size):
        self.frame_size = frame_size
        self.frame_parameter_number =  frame_parameter_number
        self.aux_number = aux_number
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        # Main input (the time frame)
        main_input = Input(shape=(frame_parameter_number,self.frame_size))
        main = Flatten()(main_input)
        
        # Additional input (other inputs)
        aux_input = Input(shape=(aux_number,))
        
        x = keras.layers.concatenate([main, aux_input])
        x = Dense(32, activation='relu')(x)
        x = Dense(32, activation='relu')(x)
        x = Dense(16, activation='relu')(x)
        out = Dense(self.action_size, activation='linear')(x)
        
        model = Model(inputs=[main_input, aux_input], outputs=out)
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        print(model.summary())
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            # The agent acts randomly
            return random.randrange(self.action_size)
        
        # Predict the reward value based on the given state
        act_values = self.model.predict(state)
        
        # Pick the action based on the predicted reward
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        # Sample minibatch from the memory
        minibatch = random.sample(self.memory, batch_size)
        
        # Extract informations from each memory
        for state, action, reward, next_state, done in minibatch:
            
            # if done, make our target reward
            target = reward
            if not done:
                # predict the future discounted reward
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
                
            # make the agent to approximately map
            # the current state to future discounted reward
            # We'll call that target_f
            target_f = self.model.predict(state)
            target_f[0][action] = target
            
            # Train the Neural Net with the state and target_f
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

Using TensorFlow backend.


## Define Trading Game

In [5]:
class TradingGame:
    def __init__(self, start_capital, frame_size, database):
        self.start_capital = start_capital
        self.frame_size = frame_size
        
        self.database = np.asarray(database["bitcoin_usd"])
        self.capital = start_capital
        self.timestep = 0
        self.time_frame = self.database[0:self.frame_size]
        
        self.position = 0
        self.state = None
    
    ### Reset function
    def reset(self):
        # Set everything to the initial start value
        self.capital = self.start_capital
        self.timestep = 0
        self.time_frame = self.database[0:self.frame_size]
        
        self.position = 0.0
        
        # state => (time_frame, capital, position)
        frame = np.reshape(self.time_frame, [1, 6, frame_size])
        aux = np.reshape([self.capital, self.position], [1, 2])
        self.state = [frame, aux]
        
        return self.state
    
    
    ### Next frame function
    def get_next_frame(self):
        self.timestep += 1
        
        if (self.frame_size+self.timestep) < self.database.shape[0]:
            next_frame = self.database[self.timestep:self.frame_size+self.timestep]
            done = False
        else:
            next_frame = self.time_frame
            done = True
            
        return next_frame, done
    
    ### Reward function
    def calc_reward(self, action, next_frame):
        # Get new price
        new_price = next_frame[self.frame_size-1, 2]
        
        # Check the action and calculate the reward
        #hold
        if action == 0: 
            reward = 0
            
        #buy
        elif action == 1: 
            if self.position is 0:
                reward = 0
                self.position = new_price
            else:
                reward = -100
                
        #sell
        elif action == 2: 
            if self.position is not 0:
                reward = new_price - self.position
                self.position = 0
            else:
                reward = -100
        return reward
    
    ### Next step function
    def step(self, action):
        # Get next time frame
        next_frame, done = self.get_next_frame()
        
        # Get reward
        reward = self.calc_reward(action, next_frame)
        
        # check if done
        self.capital += reward
        if self.capital <= 0:
            done = True
            
        # Next state
        next_frame = np.reshape(next_frame, [1, 6, frame_size])
        next_aux = np.reshape([self.capital, self.position], [1, 2])
        next_state = [next_frame, next_aux]
        
        # update state
        self.state = next_state
        
        return next_state, reward, done
    
    def get_score(self):
        return self.capital

## Train the agent

### Set parameters

In [6]:
# Parameter for Agent
frame_size = 100 # Time frame
frame_parameter_number = 6
aux_number = 2 # Additional parameters
#state_attributes = 8
action_size = 3 # hold, buy, sell

# Parameter for Trading
batch_size = 64
EPISODES = 1000
CAPITAL = 10000
#POSITION = 0

### Training

In [7]:
# Create agent
agent = DQNAgent(frame_size, frame_parameter_number, aux_number, action_size)

# Initialize Environment
env = TradingGame(CAPITAL, frame_size, DB)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 6, 100)        0                                            
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 600)           0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 2)             0                                            
____________________________________________________________________________________________________
concatenate_1 (Concatenate)      (None, 602)           0                                            
___________________________________________________________________________________________

In [8]:
#Episodes to train
for e in range(EPISODES):
    
    #Create initial state from time frame
    state = env.reset()
    #state = np.asarray(state)
    #state = np.reshape(state, [1, 6, frame_size,])

    #time frames
    for time_frame in range(frame_size, 1000):
        
        #Agent takes action
        action = agent.act(state)

        #Calc reward
        next_state, reward, done = env.step(action)
        #print(next_state)
        #next_state = np.reshape(next_state, [1, 6, frame_size])
        
        #Remember action
        agent.remember(state, action, reward, next_state, done)
        
        #Override state with next state
        state = next_state
        
        if done:
            print("episode: {}/{}, score: {}, e: {:.2}"
                  .format(e, EPISODES, env.get_score(), agent.epsilon))
            break
       
    #Replay
    if len(agent.memory) > batch_size:
        agent.replay(batch_size)
    # if e % 10 == 0:
    #     agent.save("./save/cartpole-dqn.h5")
        
    

episode: 0/1000, score: -764.8797739199999, e: 1.0
episode: 1/1000, score: -1315.9074214400025, e: 0.99
episode: 2/1000, score: -61.6176063100047, e: 0.99
episode: 3/1000, score: -75.23200641000949, e: 0.99
episode: 4/1000, score: -38.5328753599988, e: 0.98
episode: 5/1000, score: -17.793998610002745, e: 0.98
episode: 6/1000, score: -156.53788763000193, e: 0.97
episode: 7/1000, score: -7.8518777299996145, e: 0.97
episode: 8/1000, score: -51.83147110999562, e: 0.96
episode: 9/1000, score: -52.58105435000107, e: 0.96
episode: 10/1000, score: -53.883957679998275, e: 0.95
episode: 11/1000, score: -37.557810969994534, e: 0.95
episode: 12/1000, score: -22.942415450004773, e: 0.94
episode: 13/1000, score: -163.92751480000152, e: 0.94
episode: 14/1000, score: -573.1096675800018, e: 0.93
episode: 15/1000, score: -66.3959260100055, e: 0.93
episode: 16/1000, score: -48.64751762000196, e: 0.92
episode: 17/1000, score: -46.79893715999788, e: 0.92
episode: 18/1000, score: -14.631270620002397, e: 0.9

episode: 155/1000, score: 2504.3572586300015, e: 0.46
episode: 156/1000, score: 5669.307146430001, e: 0.46
episode: 157/1000, score: 8611.640524439998, e: 0.46
episode: 158/1000, score: 13057.738583859998, e: 0.45
episode: 159/1000, score: 12024.15454546999, e: 0.45
episode: 160/1000, score: 10083.476770969997, e: 0.45
episode: 161/1000, score: 7312.3831097799975, e: 0.45
episode: 162/1000, score: 11929.203248590002, e: 0.44
episode: 163/1000, score: 14145.226267610002, e: 0.44
episode: 164/1000, score: 12061.906258190002, e: 0.44
episode: 165/1000, score: 13686.710218870003, e: 0.44
episode: 166/1000, score: 6383.823340869998, e: 0.44
episode: 167/1000, score: 7758.215081410002, e: 0.43
episode: 168/1000, score: 11468.163241680008, e: 0.43
episode: 169/1000, score: 16339.110571139998, e: 0.43
episode: 170/1000, score: 4493.982479379998, e: 0.43
episode: 171/1000, score: 11983.321893399996, e: 0.42
episode: 172/1000, score: 14839.602872960002, e: 0.42
episode: 173/1000, score: 8413.745

episode: 312/1000, score: 20193.21277621, e: 0.21
episode: 313/1000, score: 18119.225756870004, e: 0.21
episode: 314/1000, score: 17619.26886181, e: 0.21
episode: 315/1000, score: 15178.7864535, e: 0.21
episode: 316/1000, score: 15708.981739689998, e: 0.21
episode: 317/1000, score: 18172.284791399998, e: 0.2
episode: 318/1000, score: 13286.305876359998, e: 0.2
episode: 319/1000, score: 20500.401650139996, e: 0.2
episode: 320/1000, score: 12341.766528480002, e: 0.2
episode: 321/1000, score: 18107.843946780005, e: 0.2
episode: 322/1000, score: 22292.382355650003, e: 0.2
episode: 323/1000, score: 20643.0943398, e: 0.2
episode: 324/1000, score: 24475.04651258, e: 0.2
episode: 325/1000, score: 15737.723524410003, e: 0.2
episode: 326/1000, score: 14437.39392281, e: 0.2
episode: 327/1000, score: 19587.628410139994, e: 0.19
episode: 328/1000, score: 20356.052714690002, e: 0.19
episode: 329/1000, score: 13647.24281551, e: 0.19
episode: 330/1000, score: 12315.9, e: 0.19
episode: 331/1000, score:

episode: 475/1000, score: 26961.37173072, e: 0.092
episode: 476/1000, score: 20911.100000000002, e: 0.092
episode: 477/1000, score: 15936.00540917, e: 0.092
episode: 478/1000, score: 23655.10076046, e: 0.091
episode: 479/1000, score: 18737.13122289, e: 0.091
episode: 480/1000, score: 17763.98760078, e: 0.09
episode: 481/1000, score: 20053.0, e: 0.09
episode: 482/1000, score: 12196.1, e: 0.089
episode: 483/1000, score: 12878.0, e: 0.089
episode: 484/1000, score: 23384.0, e: 0.088
episode: 485/1000, score: 20958.676489600002, e: 0.088
episode: 486/1000, score: 15318.371730719999, e: 0.088
episode: 487/1000, score: 25899.699999999997, e: 0.087
episode: 488/1000, score: 20772.6, e: 0.087
episode: 489/1000, score: 15737.08436781, e: 0.086
episode: 490/1000, score: 14071.009741009999, e: 0.086
episode: 491/1000, score: 18033.70759616, e: 0.085
episode: 492/1000, score: 17769.858831240002, e: 0.085
episode: 493/1000, score: 22481.6, e: 0.084
episode: 494/1000, score: 20880.657258630003, e: 0.

episode: 648/1000, score: 23235.0, e: 0.039
episode: 649/1000, score: 28411.0, e: 0.039
episode: 650/1000, score: 27349.0, e: 0.038
episode: 651/1000, score: 26880.0, e: 0.038
episode: 652/1000, score: 23321.0, e: 0.038
episode: 653/1000, score: 18504.3, e: 0.038
episode: 654/1000, score: 25404.0, e: 0.038
episode: 655/1000, score: 22130.0, e: 0.038
episode: 656/1000, score: 16869.9, e: 0.037
episode: 657/1000, score: 23231.508036029998, e: 0.037
episode: 658/1000, score: 26860.0, e: 0.037
episode: 659/1000, score: 19956.26817862, e: 0.037
episode: 660/1000, score: 19125.0, e: 0.037
episode: 661/1000, score: 20084.0, e: 0.036
episode: 662/1000, score: 20120.16087818, e: 0.036
episode: 663/1000, score: 16836.3, e: 0.036
episode: 664/1000, score: 27609.4056602, e: 0.036
episode: 665/1000, score: 16554.9, e: 0.036
episode: 666/1000, score: 21617.0, e: 0.035
episode: 667/1000, score: 17102.5, e: 0.035
episode: 668/1000, score: 17582.0, e: 0.035
episode: 669/1000, score: 23328.0, e: 0.035
e

episode: 829/1000, score: 20509.0, e: 0.016
episode: 830/1000, score: 20294.64323383, e: 0.016
episode: 831/1000, score: 23717.91563219, e: 0.016
episode: 832/1000, score: 22207.0, e: 0.015
episode: 833/1000, score: 18347.20067355, e: 0.015
episode: 834/1000, score: 19898.0, e: 0.015
episode: 835/1000, score: 21736.0, e: 0.015
episode: 836/1000, score: 16660.3, e: 0.015
episode: 837/1000, score: 22975.0, e: 0.015
episode: 838/1000, score: 19770.0, e: 0.015
episode: 839/1000, score: 19863.0, e: 0.015
episode: 840/1000, score: 23147.0, e: 0.015
episode: 841/1000, score: 21061.0, e: 0.015
episode: 842/1000, score: 18942.1, e: 0.015
episode: 843/1000, score: 22088.12051678, e: 0.015
episode: 844/1000, score: 17538.0, e: 0.015
episode: 845/1000, score: 20808.0, e: 0.014
episode: 846/1000, score: 23283.0, e: 0.014
episode: 847/1000, score: 24448.21734077, e: 0.014
episode: 848/1000, score: 27400.0, e: 0.014
episode: 849/1000, score: 18511.0, e: 0.014
episode: 850/1000, score: 20742.09025899,

def calc_next_state(state):
#Calc Capital (action == 0 then sell, action == 1 then buy) 5 coins * state_close_price
        #Calc position (current position -/+ differ from action)
        #Get newest state item and append capital and position
        #Remove latest entry and add new state item